In [135]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import re

In [211]:
driver = webdriver.Chrome(r"C:\Users\hp\chromedriver.exe")

In [292]:
url="https://www.volunteermbc.org/Listings.php?ListType=VolunteerPositionsAll&MenuItemID=1"

In [293]:
web="https://www.volunteermbc.org/"

In [294]:
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

In [295]:
bookmarks = soup.find_all('select', attrs={'name':'Bookmark'})
for bookmark in bookmarks:
    total_num = [bm.get_text() for bm in bookmark]
    print(len(total_num))

15


In [296]:
i=0
url="https://www.volunteermbc.org/Listings.php?ListType=VolunteerPositionsAll&MenuItemID=1"
pgLnk=[url]
while i < len(total_num):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    pglink = soup.find_all('a', attrs={'class':'pgLnk pgLnkNum'})
    for l in pglink:
        w = web+l.get('href')
        if w not in pgLnk and len(w)>90:
            pgLnk.append(w)
    url = pgLnk[-1]
    i = len(pgLnk)

Get main info

In [297]:
def get_main_info(mlist,Position, Organization, Location):
    for i in range(len(mlist)):
        pos = mlist[i].find('div',attrs={"class":"cell1"})
        Position.append(pos.get_text())
        org = mlist[i].find_all('div',attrs={"class":"cell2"})
        Organization.append(org[0].get_text().split(":")[1])
        Location.append(org[1].get_text().split(":")[1])
    return Position, Organization, Location

In [298]:
def get_websites(soup):
    webs = soup.find_all('span', attrs={"class": "customLink"})
    for i in range(len(webs)):
        w = webs[i].find_all("a")
        for link in w:
            Websites.append(web+link.get("href"))
    return Websites

In [405]:
def get_content(websites):
    Description=[]
    Goal=[]
    Benefits=[]
    Qualifications=[]
    Time=[]
    Training=[]
    Area_interest=[]
    for link in Websites:
        html_content = requests.get(link).text
        soup = BeautifulSoup(html_content, "lxml")
        blocks = soup.find_all('td', attrs={"class": "reportEntryValue reportEntryValuetest2"})
    
        for i in range(len(blocks)):
            if len(blocks[i].get_text()) > 0:
                content = blocks[i].get_text()
                if re.match(r"^\sDescription", content):
                    Description += [content if len(content)!=0 else "nan"]
                elif re.match(r"^\sGoal", content):
                    Goal += [content if len(content)!=0 else "nan"]
                elif re.match(r"^\sBenefits", content):
                    Benefits += [content if len(content)!=0 else "nan"]
                elif re.match(r"^\sPosition", content):
                    Qualifications += [content if len(content)!=0 else "nan"]
                elif re.match(r"^\sTime", content):
                    Time += [content if len(content)!=0 else "nan"]
                elif  re.match(r"^\sTrain", content):
                    Training += [content if len(content)!=0 else "nan"]
                elif  re.match(r"^\sArea", content):
                    Area_interest += [content if len(content)!=0 else "nan"]

        while len(Goal)<len(Description):
            Goal+=["nan"]
        assert len(Goal) == len(Description)
        while len(Training)<len(Description):
            Training+=["nan"]
        assert len(Training) == len(Description)
                
    return Description, Goal, Benefits, Qualifications, Time, Training, Area_interest

In [401]:
Position=[]
Location=[]
Organization=[]
Websites=[]
Description=[]
Goal=[]
Benefits=[]
Qualifications=[]
Time=[]
Training=[]
Area_interest=[]

In [406]:
for i in range(len(pgLnk)):
    html_content = requests.get(pgLnk[i]).text
    soup = BeautifulSoup(html_content, "lxml")
    mainlist = soup.find_all('div',attrs={'class': 'MainList'})
    Position, Organization, Location = get_main_info(mainlist, Position, Organization, Location)
    Websites = get_websites(soup)
    Description, Goal, Benefits, Qualifications, Time, Training, Area_interest = get_content(Websites)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [408]:
print("Position:{}".format(len(Position)))
print("Organization:{}".format(len(Organization)))
print("Location:{}".format(len(Location)))
print("Websites:{}".format(len(Websites)))
print("Description:{}".format(len(Description)))
print("Goal:{}".format(len(Goal)))
print("Benefits:{}".format(len(Benefits)))
print("Qualifications:{}".format(len(Qualifications)))
print("Time:{}".format(len(Time)))
print("Training:{}".format(len(Training)))
print("Area_interest:{}".format(len(Area_interest)))

Position:158
Organization:158
Location:158
Websites:158
Description:158
Goal:158
Benefits:158
Qualifications:158
Time:158
Training:158
Area_interest:158


Location:
<br> M: Mississauga
<br> B: Brampton
<br> C: Caledon
<br> V: Virtual

In [409]:
df = pd.DataFrame({'Position':Position,'Location':Location, 
                 'Organization':Organization,
                  "Websites": Websites,
                  "Description": Description,
                  "Goal": Goal, "Benefits":Benefits,
                  "Qualifications":Qualifications,
                  "Time":Time, "Training":Training, "Interest": Area_interest}) 

In [410]:
df

,Position,Location,Organization,Websites,Description,Goal,Benefits,Qualifications,Time,Training,Interest
0,Breakfast Food Ambassador,Brampton,Regeneration Outreach Community,https://www.volunteermbc.org/report.php?ListTy...,\nDescriptionWe are looking for morning people...,\nGoalWe help meet the nutritional needs of th...,\nBenefitsIf you want to feel like you are par...,\nPosition QualificationsAll you need is a pol...,\nTime Commitment Duration:Any Duration Monday...,\nTrainingOur next orientation is November 8 a...,"\nArea(s) of Interest: Food Services, Shelters..."
1,Program Assistant Volunteer (Arabic Speaking),Mississauga,Seniors Life Enhancement Centres,https://www.volunteermbc.org/report.php?ListTy...,\nDescriptionTasks for Program Assistant Volun...,\nGoalThe goal of a Program Assistant voluntee...,\nBenefitsVolunteers in this position will ben...,\nPosition QualificationsMust be able to speak...,\nTime Commitment Duration:Long Term (More tha...,\nTrainingVolunteers will be given a full orie...,\nArea(s) of Interest: Senior Support\r\n
2,Garden/Landscape Volunteer,Caledon,Bethell Hospice,https://www.volunteermbc.org/report.php?ListTy...,\nDescriptionAdopt a garden bed to maintain th...,\nGoalHospice volunteers are highly respected ...,\nBenefitsVolunteer in a team environmentApply...,\nPosition Qualifications- Minimum 18 years of...,\nTime Commitment Duration:Medium Term (4 - 6 ...,\nTrainingHospice Lite Training provided by Be...,"\nArea(s) of Interest: Environment, Seasonal/S..."
3,In-Home Visiting Volunteer,M/B/C/V,Bethell Hospice,https://www.volunteermbc.org/report.php?ListTy...,\nDescription- Uphold the client's right to di...,\nGoalHospice volunteers are highly respected ...,\nBenefitsCare for those in your communityAppl...,\nPosition Qualifications- Minimum 18 years of...,\nTime Commitment Duration:Long Term (More tha...,\nTrainingHospice Palliative Care Ontario trai...,"\nArea(s) of Interest: Healthcare, Life Skills..."
4,Resident Support Volunteer,Caledon,Bethell Hospice,https://www.volunteermbc.org/report.php?ListTy...,\nDescription- Communicate with care team for ...,\nGoalHospice volunteers are highly respected ...,"\nBenefitsWork in a team atmosphere, develop f...",\nPosition Qualifications- Minimum 18 years of...,\nTime Commitment Duration:Long Term (More tha...,\nTrainingHospice Palliative Care Ontario trai...,"\nArea(s) of Interest: Healthcare, Life Skills..."
5,Reception/ Hospitality Volunteer,M/B/C/V,Bethell Hospice,https://www.volunteermbc.org/report.php?ListTy...,\nDescription- Provide a welcoming greeting to...,\nGoalVolunteers will be located at the recept...,\nBenefits- Join in caring for those in your c...,\nPosition Qualifications- Minimum 18 years of...,\nTime Commitment Duration:Long Term (More tha...,\nTrainingHospice Lite Training in Fall 2016. ...,"\nArea(s) of Interest: Office Administration, ..."
6,Food Bank Volunteer (Malton),Mississauga,Seva Food Bank,https://www.volunteermbc.org/report.php?ListTy...,\nDescription• Welcome clients• Assist clients...,\nGoalAssist Clients with their Personal Shopp...,\nBenefitsHelping community members Customer's...,\nPosition Qualifications-Will be trained on C...,\nTime Commitment Duration:Long Term (More tha...,\nTrainingOrientation ; Training-on-the-job ; ...,"\nArea(s) of Interest: Crisis, Distress & Emer..."
7,Childcare Volunteer,Brampton,Boys and Girls Club of Peel,https://www.volunteermbc.org/report.php?ListTy...,\nDescriptionA great opportunity for a Early C...,nan,\nBenefits-Experience working with children in...,\nPosition Qualifications- A strong passion fo...,\nTime Commitment Duration:Medium Term (4 - 6 ...,nan,\nArea(s) of Interest: Parent & Child Support\r\n
8,Project Assistant - Care for Caregivers Program,Brampton,Elder Help-Peel,https://www.volunteermbc.org/report.php?ListTy...,\nDescriptionDuties of the role:• Facilitate v...,\nGoalIn cooperation with the Executive Direct...,\nBenefitsBenefits:• Gain experience in projec...,\nPositio

In [411]:
df.to_csv('MBC_volunteer.csv', index=False, encoding='utf-8_sig')

In [407]:
# Position=[]
# Location=[]
# Organization=[]
# Websites=[]
# Description=[]
# Goal=[]
# Benefits=[]
# Qualifications=[]
# Time=[]
# Training=[]
# Area_interest=[]
# for i in range(13,14):
#     html_content = requests.get(pgLnk[i]).text
#     soup = BeautifulSoup(html_content, "lxml")
#     mainlist = soup.find_all('div',attrs={'class': 'MainList'})
#     Position, Organization, Location = get_main_info(mainlist, Position, Organization, Location)
#     Websites = get_websites(soup)
#     Description=[]
#     Goal=[]
#     Benefits=[]
#     Qualifications=[]
#     Time=[]
#     Training=[]
#     Area_interest=[]
#     for link in Websites[8:9]:
#         html_content = requests.get(link).text
#         soup = BeautifulSoup(html_content, "lxml")
#         blocks = soup.find_all('td', attrs={"class": "reportEntryValue reportEntryValuetest2"})
        
#         for i in range(len(blocks)):
#             if len(blocks[i].get_text()) > 0:
#                 content = blocks[i].get_text()

#                 if re.match(r"^\sDescription", content):
#                     Description += [content if len(content)!=0 else "nan"]
# #                     print("Description:{}".format(Description[:10]))
#                 elif re.match(r"^\sGoal", content):
#                     Goal += [content if len(content)!=0 else "nan"]
# #                     print("Goal:{}".format(Goal))
#                 elif re.match(r"^\sBenefits", content):
#                     Benefits += [content if len(content)!=0 else "nan"]
# #                     print("Benefits:{}".format(Benefits[:10]))
#                 elif re.match(r"^\sPosition", content):
#                     Qualifications += [content if len(content)!=0 else "nan"]
# #                     print("Benefits:{}".format(Benefits[:10]))
#                 elif re.match(r"^\sTime", content):
#                     Time += [content if len(content)!=0 else "nan"]
# #                     print("Time:{}".format(Time[:10]))
#                 elif  re.match(r"^\sTrain", content):
#                     Training += [content if len(content)!=0 else "nan"]
# #                     print("Training:{}".format(Training[:10]))
#                 elif  re.match(r"^\sArea", content):
#                     Area_interest += [content if len(content)!=0 else "nan"]
# #                     print("Area_interest:{}".format(Area_interest[:10]))
#                 else:
#                     Description +=["nan"]
#                     Goal +=["nan"]
#                     Benefits +=["nan"]
#                     Qualifications +=["nan"]
#                     Time +=["nan"]
#                     Training +=["nan"]
#                     Area_interest +=["nan"]
        
                